A simple web scraper that seaches stock 

In [20]:
import pytest
from selenium.webdriver import Chrome
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


driver = Chrome(executable_path='/Users/test/Desktop/RPA/RPA/selenium/chromedriver')
driver.implicitly_wait(10)
driver.get("https://www.google.com")

# set the keyword you want to search for
keyword = 'stocks' 
# we find the search bar using it's name attribute value
searchBar = driver.find_element_by_name('q')
# first we send our keyword to the search bar followed by the enter # key
searchBar.send_keys(keyword)
searchBar.send_keys('\n')

time.sleep(10)

def scrape():
    pageInfo = []
    try:
      # wait for search results to be fetched
       WebDriverWait(driver, 10).until(
       EC.presence_of_element_located((By.CLASS_NAME, "g"))
      )
    except Exception as e:
        print(e)
        driver.quit()
   # contains the search results
    searchResults = driver.find_elements_by_class_name('g')

   #aCOpRe
    for result in searchResults:
        element = result.find_element_by_css_selector('a') 
        link = element.get_attribute('href')
        header = result.find_element_by_css_selector('h3').text
        text = result.find_element_by_class_name('aCOpRe').text        
        pageInfo.append({
          'header' : header, 'link' : link, 'text': text
        })
    return pageInfo

# Number of pages to scrape
numPages = 50
# All the scraped data
infoAll = []
# Scraped data from page 1
infoAll.extend(scrape())
for i in range(0 , numPages - 1):
    nextButton = driver.find_element_by_link_text('Next')
    nextButton.click()
    infoAll.extend(scrape())


df = pd.DataFrame(infoAll)
fileName = keyword + '_' + str(numPages) + '.csv'
df.to_csv(fileName)
df = df.iloc[5:]

In [21]:
df.head(10)

,header,link,text
5,Stock Market Data with Stock Price Feeds | Nasdaq,https://www.nasdaq.com/market-activity/stocks,"Find stock quotes, interactive charts, histori..."
6,"Here are analysts' favorite marijuana stocks, ...",https://www.marketwatch.com/story/here-are-ana...,4 hours ago — Marijuana stock list · The ETFMG...
7,Stock investment news - MarketWatch,https://www.marketwatch.com/investing/stocks,"News, analysis and advice on investing in stocks."
8,Stocks | Investor.gov,https://www.investor.gov/introduction-investin...,What are stocks? Stocks are a type of security...
9,Find the Best Stocks to Buy Today | US News Be...,https://money.usnews.com/investing/stocks,Find new investments and research thousands of...
10,How To Invest In Stocks - The Balance,https://www.thebalance.com/stocks-4073971,Investing in the stock market is the most trie...
11,Stocks: What Are They? - The Balance,https://www.thebalance.com/what-are-stocks-330...,Stocks represent ownership in a publicly-trade...
12,Stock Market News - CNBC,https://www.cnbc.com/stocks/,"10 hours ago — In-depth market analysis, real-..."
13,The 21 Best Stocks to Buy for 2021 | Kiplinger,https://www.kiplinger.com/investing/stocks/601...,The stock market always has a few surprises in...
14,Definition: What Is Stock? - NerdWallet,https://www.nerdwallet.com/article/investing/w...,Definition: What are stocks? Stocks are securi...
